# Домашнее задание №4. Тематическое моделирование. EM-алгоритм

In [14]:
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
from pymystem3 import Mystem
import nltk
nltk.download('stopwords')
import pymorphy2
from gensim.models import *
from gensim import corpora
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
import warnings
def fxn():
    warnings.warn("deprecated", DeprecationWarning)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\BazhanovaEN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Задание
продолжаем исследовать датасет с твитами.

1. Объединить в одну выборку (положительные и отрицательные твиты).

2. Провести исследование и выявить тематики, о которых говорят в твитах.

3. Сделать визуализацию кластеров тематик.

4. Проинтерпритировать получившиеся тематики

## План решения

[0. Загрузка и просмотр данных](#section_0)

[1. Предобработка данных](#section_1)

[2. Модель Latent Dirichlet Allocation (LDA)](#section_2)

[3. Визуализация предсказаний модели LDA](#section_3)

[4. Выводы (определение тематик в твитах)](#section_4)

### 0. Загрузка и просмотр данных <a id='section_0'></a>

Загрузим данные и объединим их в один датасет.

In [15]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('positive(2).csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)

negative = pd.read_csv('negative(2).csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)

df = pd.concat([positive, negative], ignore_index=True)
df.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


In [16]:
#выведем информацию о датасете
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226834 entries, 0 to 226833
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    226834 non-null  object
 1   label   226834 non-null  object
dtypes: object(2)
memory usage: 3.5+ MB


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### 1. Предобработка данных <a id='section_1'></a>

In [17]:
words_regex = re.compile('\w+') #компилируем объект (1 или более букв)

# составляем список токенов из букв и длины >3
def find_words(text, regex = words_regex):
    tokens = regex.findall(text) #ищем все слова в тексте --> получаем список токенов
    return [w for w in tokens if w.isalpha() and len(w)>= 4] #берем токеты из списка токетов, если они состоят только из букв и имеют длину >3

morph = pymorphy2.MorphAnalyzer() #морфологический анализ
stopwords_list = stopwords.words('russian') #список стоп-слов

# составляем списка нормальных форм токенов, убираем стопслова и токены с небуквами 
def lemmatize(words, lemmer = morph, stopwords = stopwords_list):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words] #список нормальных форм для всех токенов
    return [w for w in lemmas if not w in stopwords and w.isalpha()] #удаляем из списка нормальных форм стоп-слова и токены с небуквой(ми)

#очищаем токены от нецензурной лексики и ненужныъ слов
def clear_tokens(tokens):
    string_tokens = ' '.join(tokens)
    string_tokens = re.sub(r'[A-za-z]', ' ',  string_tokens) 
    string_tokens = re.sub(r'\w+ще\b', ' ',  string_tokens) 
    string_tokens = re.sub(r'\bбл\w+', ' ',  string_tokens) 
    string_tokens = re.sub(r'значит', ' ',  string_tokens) 
    string_tokens = re.sub(r'почему', ' ',  string_tokens) 
    string_tokens = re.sub(r'свой', ' ',  string_tokens) 
    string_tokens = re.sub(r'капец', ' ',  string_tokens) 
    string_tokens = re.sub(r'который', ' ',  string_tokens)
    string_tokens = re.sub(r'это', ' ',  string_tokens)
    string_tokens = re.sub(r'весь', ' ',  string_tokens)
    string_tokens = re.sub(r'\bидио\w+', ' ',  string_tokens) 
    string_tokens = re.sub(r'\bпид\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bхуй\w+', ' ', string_tokens)
    string_tokens = re.sub(r'\w+xу.\b', ' ',  string_tokens)
    string_tokens = re.sub(r'\w+xу\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\w+но\b', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпоп\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпиз\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bгов\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bфиг\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'сука', ' ',  string_tokens)
    string_tokens = re.sub(r'очень', ' ',  string_tokens)
    string_tokens = re.sub(r'лох', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпи.де\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'нету', ' ',  string_tokens)
    string_tokens = re.sub(r'ой', ' ',  string_tokens)
    string_tokens = re.sub(r'\bхот\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпрост\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bника\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\w+ть\b', ' ',  string_tokens)
    string_tokens = re.sub(r'мочь', ' ',  string_tokens)
    clear_tokens = string_tokens.split()
    clear_tokens = [w for w in clear_tokens if len(w)>2]
    return clear_tokens

def preprocess(text):
    return (clear_tokens(lemmatize(find_words(text))))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:1: DeprecationWarning: invalid escape sequence \w
<>:1: DeprecationWarning: invalid escape sequence \w
<ipython-input-17-a5cb4e807e76>:1: DeprecationWarning: invalid escape sequence \w
  words_regex = re.compile('\w+') #компилируем объект (1 или более букв)
C:\ProgramData\Anaconda3\lib\site-packages\pymorphy2\units\base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
C:\ProgramData\Anaconda3\lib\site-packages\pymorphy2\units\base.py:70: DeprecationWarning: inspect.getargspec() is deprec

Применяем предобработку к твитам

In [18]:
df['text'] = df['text'].apply(preprocess)
df.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,text,label
0,"[школотый, поверь, самый, общество, предмет, тип]",positive
1,"[таки, немного, похожий, мальчик, хороший]",positive
2,[испугаться],positive
3,"[угол, голод, порция]",positive
4,"[страшилка, создаться, ощущение, автор]",positive


### 2. Модель Latent Dirichlet Allocation (LDA) <a id='section_2'></a>

**Создадим словарь из всех уникальных токенов датасета**

In [19]:
dictionary = corpora.Dictionary(df['text']) #создаем словарь между токенами и их индексами (на вход список списков)

dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n=None) # игнорируем слова, которые встречаются реже 10 раз или составляют более 0.9 словаря 

dictionary.save('tweet.dict')

len(dictionary.keys()) #длина словаря из токенов

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


8172

**Векторизуем твиты**

In [20]:
corpus = [dictionary.doc2bow(text) for text in df['text']] #для каждого списка токенов возвращает кортеж (индекс токена, количество встречаний токена в твите)
corpora.MmCorpus.serialize('tweet.model', corpus) #выполняем итерацию по корпусу (записываем смещение байта каждого твита). Полученную структуру индексов сохраняем в файл

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
#теперь по словарю можно итерироваться
mm = corpora.MmCorpus('tweet.model') # корпус имеет произвольный доступ
print(df['text'].values[0])
print(mm[0]) #  извлекаем документ под номером 0

['школотый', 'поверь', 'самый', 'общество', 'предмет', 'тип']
[(0, 1.0), (1, 1.0), (2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0)]


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**Обучаем модель**

In [22]:
lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=3)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**Определяем вероятные темы**

In [23]:
lda.show_topics(num_topics=3, num_words=20, formatted=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.035*"сегодня" + 0.028*"завтра" + 0.027*"день" + 0.022*"дом" + 0.018*"школа" + 0.011*"настроение" + 0.010*"первый" + 0.010*"работа" + 0.010*"урок" + 0.008*"прийти" + 0.008*"найти" + 0.007*"друг" + 0.007*"каникулы" + 0.007*"втор" + 0.007*"улица" + 0.007*"снег" + 0.005*"учёба" + 0.005*"город" + 0.005*"уйти" + 0.005*"конец"'),
 (1,
  '0.020*"больш" + 0.017*"новый" + 0.010*"человек" + 0.010*"год" + 0.010*"неделя" + 0.009*"скоро" + 0.009*"время" + 0.009*"голова" + 0.009*"мама" + 0.008*"день" + 0.008*"телефон" + 0.008*"нравиться" + 0.007*"бояться" + 0.007*"закончиться" + 0.007*"вроде" + 0.007*"рука" + 0.006*"спасибо" + 0.006*"мало" + 0.006*"слово" + 0.006*"ужасный"'),
 (2,
  '0.020*"хороший" + 0.016*"жизнь" + 0.016*"человек" + 0.014*"самый" + 0.014*"идти" + 0.014*"никто" + 0.013*"утро" + 0.013*"жаль" + 0.010*"ночь" + 0.008*"экзамен" + 0.008*"любимый" + 0.008*"снова" + 0.008*"дело" + 0.007*"жалко" + 0.007*"казаться" + 0.007*"час" + 0.007*"прийтись" + 0.007*"фильм" + 0.007*"пока" + 0.

**Оценки качества полученной модели**

In [24]:
print(lda.log_perplexity(corpus))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


-8.008838212350929


In [25]:
print('Персплексия: ', np.exp(lda.log_perplexity(corpus)))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Персплексия:  0.000332510694801722


### 3. Визуализация предсказаний модели LDA <a id='section_3'></a>

In [26]:
vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


### 4. Выводы (определение тематик в твитах) <a id='section_4'></a>

На основе резульатов работы модели можно сформулировать следующие примерные названия тем твитов:  
    
**Тема 1.** Работа/Образование.

**Тема 2.** Чувства и эмоции (личные отношения).

**Тема 3.** Время (планирование деятельности).

P.S. Количество тем, равное 3, является оптимальным. При большем количестве тем, они будут слищком "близко" располагаться друг к другу.